<a href="https://colab.research.google.com/github/karoz-exe/Homework-Kazancev-R.A-/blob/hw_10/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Практическое задание №10**

В этом задании вам предстоит обучить модель для прогнозирования стоимости недвижимости и создать API для модели с помощью FastAPI.

Вам нужно:
1. Скачать данные и обучить модель прогнозирования стоимости недвижимости (модель может быть любой сложности, даже линейная регрессия на двух признаках. Можно переиспользовать модель с прошлого ДЗ) - 1 балл
2. Реализуйте код для получения предсказания обученной моделью - 1 балл
Реализуйте получение предсказания моделью через get-запрос по адресу /predict_get - 2 балла
3. Реализуйте получение предсказания моделью через post-запрос по адресу /predict_post - 2 балла
4. Реализуйте liveness-пробу (health-check) health - 1 балл
5. Запустите API через uvicorn, посетите адрес http://127.0.0.1:8000/docs и попробуйте отправить get/post запросы к вашему API через интерфейс Swagger-документации. Результаты выполнения запросов сохраните в виде скриншотов в репозитории - 3 балла

In [ ]:
!pip install uvicorn fastapi scikit-learn joblib pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.5 MB/s eta 0:00:00


In [ ]:
import uvicorn
from fastapi import FastAPI, HTTPException
from fastapi.responses import JSONResponse
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import joblib
import os

app = FastAPI()

MODEL_PATH = "linear_model.pkl"

# --- Загрузка и обучение модели (выполняется только один раз при запуске) ---
if not os.path.exists(MODEL_PATH):
    try:
        data = pd.read_csv('/content/realty_data.csv')
        data = data.dropna(subset=['price', 'floor', 'total_square']) #Удаление строк с пропусками
        X = data[['floor', 'total_square']]
        y = data['price']
        model = LinearRegression()
        model.fit(X, y)
        joblib.dump(model, MODEL_PATH)
        print("Модель обучена и сохранена.")
    except FileNotFoundError:
        print("Файл realty_data.csv не найден. Модель не обучена.")
        model = None
    except Exception as e:
        print(f"Ошибка при обучении модели: {e}")
        model = None
else:
    try:
        model = joblib.load(MODEL_PATH)
        print("Модель загружена из файла.")
    except Exception as e:
        print(f"Ошибка при загрузке модели: {e}")
        model = None


# --- Эндпоинты ---
@app.get("/health")
async def health_check():
    return {"status": "healthy"}

@app.get("/predict_get")
async def predict_get(floor: float, total_square: int):
    if model is None:
        raise HTTPException(status_code=500, detail="Модель не загружена.")
    try:
        prediction = model.predict([[floor, total_square]])[0]
        return {"price": prediction}
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Ошибка прогнозирования: {e}")

@app.post("/predict_post")
async def predict_post(data: dict):
    if model is None:
        raise HTTPException(status_code=500, detail="Модель не загружена.")
    try:
        floor = float(data["floor"])
        total_square = int(data["total_square"])
        prediction = model.predict([[floor, total_square]])[0]
        return {"price": prediction}
    except KeyError as e:
        raise HTTPException(status_code=422, detail=f"Отсутствует обязательное поле: {e}")
    except ValueError as e:
        raise HTTPException(status_code=422, detail=f"Неверный формат данных: {e}")
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Ошибка прогнозирования: {e}")

Модель загружена из файла.


In [ ]:
!pip install FastAPI -q
!uvicorn main:app --reload
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.1", port=8000)

INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [30660] using StatReload
INFO:     Started server process [30662]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
